# 0.0.Entendimento Do Problema de Negócio

- Objetivo do Problema
    - 1.0.Previsão do preço final para cada casa.
- Por que?
    - Qual o Tipo de modelo Negocio que estamos lidando?
        - Estamos lidando com um tipo de négocio de compra e venda de casas.
           - Comprar casas por um preço menor e vender por um preço mais caro
    - Como a empresa ganha dinheiro?(A grosso modo)
        - A empresa ganha dinheiro comprando casas por um preço menor e vendendo por um preço maior.
            - Receita: Venda - Compra = Receita.
            
- Contexto:
    - Possivelmente o problema aqui e a automação da precificação dos preços das casas, pois a empresa está querendo comprar mais imóveis e não sabe o preço que aquele imóvel deveria estar, e por esse motivo acaba deixando passar boas oportunidades de compra ou até comprando imóveis que estão com o preço além do que deveria estar.
           
- Proposta de Solução
    - 1.0.Para descobrir o preço justo da casa eu vou fazer um modelo preditivo que dada as caracteristicas nos de o preço do imóvel.
    - 2.0.Como o modelo vai ajudar a empresa a ganhar mais dinheiro?
        - O modelo vai ajudar na hora de comprar um imóvel, como ele vai nos trazer a precificação que aquele imóvel deveria estar, o comprador vai saber exatamente se está comprando um imóvel que está com o preço abaixo ou acima do que deveria estar, tendo essa informação em mãoes ele vai poder negociar de uma melhor forma a compra de um imóvel, afetando assim diretamente a Receita da empresa, pois ele vai saber as melhores oportunidades de compra, consequentemente comprando mais barato e no futuro vendendo mais caro, assim aumentando a Receita da empresa.
    - 2.0.Produto Final(O que eu vou entregar no fim do projeto)
       - WebApp
         - Input: Recebe as caracteristicas da casa
         - Output: Retorna o preço da casa.

# 1.0.Imports

In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
#Show all columns
pd.set_option('display.max_columns', None)

#filter warnings
import warnings
warnings.filterwarnings('ignore')

## 1.1.Load Data

In [2]:
df_train = pd.read_csv('datasets/train.csv')

## 1.2.Help Functions

In [46]:
def cross_validation(nome_modelo,model,num_folds,x_train,y_train,verbose=False):
    r2_list  = []
    rmse_list   = []
    mae_list    = []
    mape_list   = []
    
    #generate k-fold
    Kfold = StratifiedKFold(n_splits=num_folds,shuffle=True,random_state=32)
    
    i=1
    for train_ix, val_ix in Kfold.split(X_train,y_train):
        if verbose:
            print('Fold Number: {}/{}'.format(i,num_folds))
        
        #get fold
        x_train_fold = X_train.iloc[train_ix]
        y_train_fold = y_train.iloc[train_ix]
        
        x_val_fold = X_train.iloc[val_ix]
        y_val_fold = y_train.iloc[val_ix]
        
        
        #Model train
        model.fit(x_train_fold,y_train_fold)

        #prediction
        pred_nn = model.predict(x_val_fold)


        #Metrics
        mae = mean_absolute_error(y_val_fold, pred_nn)
        mae_list.append(mae)

        mape = mean_absolute_percentage_error(y_val_fold,pred_nn)
        mape_list.append(mape)

        r2 = r2_score(y_val_fold,pred_nn)
        r2_list.append(r2)

        rmse = np.sqrt(mean_squared_error(y_val_fold,pred_nn))
        rmse_list.append(rmse)

        i+=1
    return pd.DataFrame({'Nome Modelo':nome_modelo,
                         'MAE CV':np.round(np.mean(mae_list),2).astype(str) +  ' +/- ' +np.round(np.std(mae_list),2).astype(str),
                         'MAPE CV':np.round(np.mean(mape_list),2).astype(str) +  ' +/- ' +np.round(np.std(mape_list),2).astype(str),
                         'RMSE CV':np.round(np.mean(rmse_list),2).astype(str) +  ' +/- ' +np.round(np.std(rmse_list),2).astype(str),
                         'R² CV':np.round(np.mean(r2_list),2).astype(str) +  ' +/- ' +np.round(np.std(r2_list),2).astype(str)},index=[0])
        
def avaliar_modelo(nome_modelo,y_test,previsao):
    mae = mean_absolute_error(y_test,previsao)
    mape = mean_absolute_percentage_error(y_test,previsao)
    r2 = r2_score(y_test,previsao)
    rmse = np.sqrt(mean_squared_error(y_test,previsao))

    return pd.DataFrame({'Model name':nome_modelo,
                         'MAE':'{:.2f}'.format(mae),
                         'MAPE':'{:.2%}'.format(mape),
                         'RMSE':'{:.2f}'.format(rmse),
                         'R²':'{:.2%}'.format(r2)},index=[0])

# 2.0.Data Descripition

In [4]:
df1 = df_train.copy()
df1.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## 2.1.Data Dimensions

In [5]:
print('Numbers of rows: {}'.format(df1.shape[0]))
print('Numbers of columns: {}'.format(df1.shape[1]))

Numbers of rows: 1460
Numbers of columns: 81


## 2.2.Data Types

In [6]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## 2.3.Change Data Type

In [7]:
#None to change

## 2.4.NA Check

**Colunas que serão deletadas:**
Colunas com 80 % ou mais de valores nulos serão deletadas, pelo motivo de estarem praticamente totalmente vazias e caso eu preencha elas com qualquer valor vou estar mudando o fenomeno que estou tentando prever.

In [8]:
aux = df1.isna().sum().reset_index().rename(columns={0:'NAN_Values','index':'Attributes'})
aux = aux[aux['NAN_Values'] > 0].sort_values(by='NAN_Values',ascending=False).reset_index(drop=True)
aux['% NA_Values'] = round(aux['NAN_Values'] / len(df1),2)
aux

,Attributes,NAN_Values,% NA_Values
0,PoolQC,1453,1.00
1,MiscFeature,1406,0.96
2,Alley,1369,0.94
3,Fence,1179,0.81
4,FireplaceQu,690,0.47
5,LotFrontage,259,0.18
6,GarageType,81,0.06
7,GarageYrBlt,81,0.06
8,GarageFinish,81,0.06
9,GarageQual,81,0.06


## 2.5.Substition Values NA

**Preenchimento de valores nulos:**
Vou escolher prencher os valores de LotFrontage e MasVnrArea com a médiana dos valores, pois depois de analisar a distribuição dessas variáveis percebi que havia uma concentração muito forte de valores entre 50 - 100 no caso da LotFrontage e na MasVnrArea o valor de zero é o que mais aparece, por isso vou escolher usar a Mediana, pois ele não sofre com valores outliers e é mais seguro em casos assim.

Já no caso da GarageYrBlt como e uma variavel de ano, eu vou escolher preencher os valores pela moda, pois em váriaveis de ano assim, não vejo problema em colocar o valor mais frequente.


**Removendo colunas e dropando valores NA das colunas categoricas:** Vou escolher remover a coluna categorica **FireplaceQu** pelo motivo de ser uma coluna categoria e possuir 690 valores categoricos faltantes, faço isso pois caso eu descida substituir o valor dessa coluna existe grande chances de eu inviezar o modelo, ou seja, fazer o modelo aprender aquilo que eu quero e não a realidade, por esse motivo vou escolher remover a coluna e as outras colunas colunas que são **Categoricas** vou escolher drop os valores.

In [9]:
cols_na = ['PoolQC','MiscFeature','Alley','Fence', #columns with NA values >= 80%
             'FireplaceQu' ] #column with 690 values categorical NA

In [10]:
#Drop columns with NA values
df1 = df1.drop(cols_na,axis=1)

#filling other columns numeric with the median
lot_median_value = df1['LotFrontage'].median()
df1['LotFrontage'] = df1['LotFrontage'].fillna(lot_median_value)

mas_vnr_area_median = df1['MasVnrArea'].median()
df1['MasVnrArea'] = df1['MasVnrArea'].fillna(mas_vnr_area_median)

garage_year_built_mode = df1['GarageYrBlt'].mode()[0]
df1['GarageYrBlt'] = df1['GarageYrBlt'].fillna(garage_year_built_mode)

#Drop NA values of categorical features
df1 = df1.dropna()

In [11]:
#rows and columns
df1.shape

(1338, 76)

# 3.0.Data Filtering

In [12]:
df2 = df1.copy()

## 3.1.Filter Rows

## 3.2.Filter Columns

In [13]:
#Drop columns with NA values
cols = ['MoSold','YrSold','SaleType','SaleCondition'] # Columns of business
df2 = df2.drop(cols,axis=1)

# 4.0.Data Preparation

In [14]:
df3 = df2.copy()

In [15]:
#dummy variables
df3_dummy = pd.get_dummies(df3.drop('SalePrice',axis=1))

#join SalePrice with variables dummys
df3 = pd.concat([df3['SalePrice'],df3_dummy],axis=1)

In [16]:
#rows and columns
df3.shape

(1338, 250)

# 5.0.Machine Learning Model

In [17]:
df4 = df3.copy()

In [18]:
X = df4.drop('SalePrice',axis=1)
y = df4['SalePrice']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=32)

In [19]:
X_train.shape,X_test.shape, y_train.shape, y_test.shape

((936, 249), (402, 249), (936,), (402,))

## BaseLine - Linear Model

In [38]:
ln = LinearRegression()
ln.fit(X_train,y_train)
previsao_bn = ln.predict(X_test)

## Random Forest

In [21]:
rf = RandomForestRegressor()
rf.fit(X_train,y_train)
previsao_rf = rf.predict(X_test)

## Random Forest - Cross Validation

In [53]:
rf_cross = RandomForestRegressor()
rf_result_cross = cross_validation('Random Forest',rf_cross,5,X_train,y_train)

## Baseline - Liner Model - Cross Validation

In [52]:
ln_cross = LinearRegression()
ln_result_cross = cross_validation('Linear Model',ln_cross,5,X_train,y_train)

# 6.0. Perfomance

## 6.1.Simple perfomance

In [57]:
baseline_result = avaliar_modelo('BaseLine - Linear Model',y_test,previsao_bn)

In [58]:
rf_result = avaliar_modelo('Random Forest Regression',y_test,previsao_rf)

In [60]:
modeling_result = pd.concat([baseline_result,rf_result])
modeling_result.sort_values('RMSE')

,Model name,MAE,MAPE,RMSE,R²
0,Random Forest Regression,17599.56,10.02%,26366.07,88.11%
0,BaseLine - Linear Model,17923.93,9.89%,28482.96,86.12%


## 6.2.Real Perfomance - Cross Validation

In [61]:
modeling_result_real = pd.concat([ln_result_cross,rf_result_cross]) 
modeling_result_real.sort_values('RMSE CV')

,Nome Modelo,MAE CV,MAPE CV,RMSE CV,R² CV
0,Random Forest,18339.9 +/- 751.9,0.1 +/- 0.01,30856.79 +/- 5408.71,0.85 +/- 0.04
0,Linear Model,20564.83 +/- 1432.38,0.12 +/- 0.01,39882.16 +/- 11366.02,0.73 +/- 0.16


# Conclusion

**O que eu foi feito nessa etapa:**
- Preenchendo e removendo valores nulos
    - Dessa vez eu analisei os valores nulos e retirei ou preenchi por um motivo já especificado.
- Criando e Aplicando a Cross validation
    - Para termos um resultado mais real e que condiz com a realidade foi necessario criar e usar a cross nesse problema.
    
**O que ainda falta fazer?**
- Relembrando nosso objetivo de prever o preço das casas e vendo que o modelo já está conseguindo ter exelentes resultados com o R² de 85%, eu já poderia escolher acabar o projeto por aqui mesmo, e ir para o deploy, porém ainda e necessario realizar alguns passos, que são:
    - Feature Enginnering: Criar mais colunas para explicar melhor o fenomeno
    - Feature Selection: Selecionar as colunas mais relevantes, pois ainda temos muuuiiitas colunas no nosso dataset e isso aumenta muito a dimensionalidade do problema e pensando em casos práticos, pode ser que a pessoa não tenha todas as 76 informações que o modelo pede na hora que ela for compra o imóvel, por causa disso ainda vamos ter que seleciona as colunas mais relevantes.
    
 
**Proximos passos:**
- Feature Enginnering.
- Análise exploratória dos dados.